### Cine21 영화배우 정보 크롤링
* site 주소 : http://www.cine21.com/rank/person
* 개발자 도구에서 xhr 인 것만 필터링 network -> content 페이지의 요청 방식 확인
* Request URL: http://www.cine21.com/rank/person/content
* Request Method : Post
* From Data (1개월치 데이터)
    - section: actor
    - period_start: 2020-07
    - gender: all
    - page: 1
* 1 ~ 2년치 데이터를 한꺼번에 크롤링 해서 actors.json 파일로 저장하기

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pymongo
from urllib.parse import urljoin

### 정규 표현식
- \w 는 문자와 숫자
<pre>
     * : 0 ~ n (횟수), + : 1 ~ n, ? : 0(zero) or 1
</pre>
<br>
<br>
- .(period) : 줄바꿈 문자(\n)을 제외한 모든 글자 1개를 의미한다.
- .* 는 문자가 0번 또는 그 이상 반복되는 패턴
<br>
<br>
- Greedy(.\*) vs Non-Greedy(.*?) ex) <span.*?>.*?</span>
    - Greedy(.\*) (욕심많은) : 모두 선택됨
    - Non-Greedy(.*?) (욕심없는) : 첫번째 매칭되는 태그만 선택됨
<br>
<br>    
- [\n\r\t] : \n, \r, \t 중 하나는 찾는다.
<br>
<br>
- re.compile(r'[\n\r\t]') 는 해당 정규표현식을 찾아주는 컴파일러를 생성해주는 함수이다.
    - compile 함수 안에 정규표현식 앞에는 항상 r 을 붙여줘야 한다.

#### 1단계

In [4]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict = dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2020-07'
formdata_dict['gender'] = 'all'
formdata_dict['page'] = 1
formdata_dict

res = requests.post(actor_url, data=formdata_dict) #post 방식에는 data속성에 넣어야함 / 통신을 위한 설정
print(res.ok) # 통신확인

soup = BeautifulSoup(res.text, 'html.parser') # html 형식을 파싱할 준비 완료
soup
actor_item_list = list() # 각 배우들을 저장할 list
 # ul > li.people_li > div.name > a
for actor in soup.select('li.people_li div.name'):
    #print(actor)
    print(re.sub('\(\w*\)','',actor.text)) # 정규 표현식 적용

# ------------------------------------------------------------------------------------------

for a_tag in soup.select('li.people_li div.name a'):
    #print(a_tag['href'])
    actor_detail_url = urljoin(actor_url, a_tag['href']) # main_ulr + 각 a_tag['href'] 의 url
    #print(actor_detail_url)

    res = requests.get(actor_detail_url)
    soup = BeautifulSoup(res.text, 'html.parser')

    actor_item_dict = dict()
    for li_tag in soup.select('ul.default_info li'): # a 태그 안에 있는 li 태그들    
        #print(li_tag)
        #print(li_tag.text)

        #dict의 key에 해당하는 값을 추출한다. dict['직업']
        actor_item_field = li_tag.select_one('span.tit').text # 직업
        #print(actor_item_field) # 키 값 확인
        
        #dict의 value에 해당하는 값을 추출한다. dict['직업'] = '배우' 를 뽑아낸다.
        #<li><span class="tit">직업</span>배우</li>
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(li_tag))  # 값들 span 안에 있는 태그들 정규 표현식으로 파싱 / #<li>배우</li>
        actor_item_value = re.sub('<.*?>', '', actor_item_value) # 배우

        """ 적용 전
        \nhttps://www.facebook.com/cjesminjae
        """

        regex = re.compile(r'[\n\r\t]')
        actor_item_value = regex.sub('', actor_item_value)

        """ 적용 후
        https://www.facebook.com/cjesminjae
        """

        actor_item_dict[actor_item_field] = actor_item_value
        #print(actor_item_dict) # 누적되어 쌓임

    actor_item_list.append(actor_item_dict)

actor_item_list

True
강동원
이정현
이레
권해효
구교환
김민재
유아인


[{'직업': '배우',
  '생년월일': '1981-01-18',
  '성별': '남',
  '홈페이지': 'http://www.gangdongwon.com',
  '신장/체중': '186cm, 68kg',
  '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)',
  '취미': '음악감상, 웹서핑, 운동, 신발 모으기',
  '특기': '축구'},
 {'직업': '배우',
  '생년월일': '1980-02-07',
  '성별': '여',
  '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh',
  '신장/체중': '158cm, 38kg',
  '학교': '중앙대학교 연극영화',
  '취미': '바비 인형 수집, 피아노 연주',
  '특기': '별관찰, 시나리오 쓰기',
  '소속사': '에이바 앤 엔터테인먼트'},
 {'직업': '배우',
  '생년월일': '2006-03-12',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/leere.vv/'},
 {'직업': '배우',
  '생년월일': '1965-11-06',
  '성별': '남',
  '신장/체중': '170cm, 62kg',
  '학교': '한양대학교 연극영화학 졸업'},
 {'직업': '배우', '생년월일': '1982-12-14', '성별': '남'},
 {'직업': '배우',
  '생년월일': '1979-03-20',
  '성별': '남',
  '홈페이지': 'https://www.facebook.com/cjesminjae',
  '학교': '한국예술종합학교'},
 {'다른 이름': '엄홍식',
  '직업': '배우',
  '생년월일': '1986-10-06',
  '성별': '남',
  '홈페이지': 'https://www.instagram.com/hongsick/https://twitter.com/seeksik',
  '신장/체중': '178cm

In [49]:
# 정규 표현식 테스트

test_data = '유아인(2편)'
re.sub('\(\w*\)','',test_data) # 정규 표현식 사용 / 앞에 '\(\)' >> ()찾아라 / '' << empty 이걸로 바꿔라

'유아인'

In [75]:
# 정규 표현식 white space 테스트

# 공백, 탭 등 white space제거
test_data = """
\n
\thttps://www.facebook.com/cjesminjae
\n
학교"""
print(test_data)
regex = re.compile(r'[\n\r\t]') # \n\r\t 이걸 찾아라
regex.sub('',test_data) # 위에 찾은 값들을 공백문자로 대체해라




	https://www.facebook.com/cjesminjae


학교


'https://www.facebook.com/cjesminjae학교'

#### 2단계
* 1 page의 7명 배우의 정보를 출력

In [101]:
# 흥행지수
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict = dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2020-07'
formdata_dict['gender'] = 'all'
formdata_dict['page'] = 1
formdata_dict

res = requests.post(actor_url, data=formdata_dict) #post 방식에는 data속성에 넣어야함 / 통신을 위한 설정
print(res.ok) # 통신확인

soup = BeautifulSoup(res.text, 'html.parser') # html 형식을 파싱할 준비 완료
soup

actor_item_list = list() # 각 배우들을 저장할 list

actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong') # '>' >> 바로 아래에 있다는 표시
movies = soup.select('ul.mov_list')
rankings = soup.select('li.people_li > span.grade') #print(len(rankings))

#actor 변수가 <div class=name> 를 의미함
for index, actor in enumerate(actors): 
    actor_item_dict = dict()
    #print(actor)
    #유아인(2편) 에서 (2편)을 제거하고 저장한다.
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_item_dict['배우이름'] = actor_name

    #흥행지수 값에서 ,(콤마)를 제거하고 정수타입으로 변환해서 저장한다.
    actor_hit = int(hits[index].text.replace(',', '')) # replace() 함수 : ',' 를 빼주기 위함
    actor_item_dict['흥행지수'] = actor_hit
    #print(artor_name, actor_hit) 배우이름, 흥행지수

    #출연작
    movie_titles = movies[index].select('li a span')
    #출연작 여러개의 title을 저장하는 리스트
    movie_title_list = list()
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    actor_item_dict['출연영화'] = movie_title_list
    
    #순위
    actor_ranking = rankings[index].text
    actor_item_dict['랭킹'] = int(actor_ranking)
    #print(artor_name, actor_ranking)

# ------------------------------------------------------------------------------------------
    #배우의 상세정보를 보기 위해 http get 요청 다시 요청함
    #('li.people_li div.name a')
    """
        <div class=name> = actors
            <a href="/db/person/info/?person_id=18040"> = actor_detail_url
    """
    actor_detail_url = actor.select_one('a').attrs['href'] #attrs[속성]
    actor_detail_full_url = urljoin(actor_url, actor_detail_url)
   #print(actor_detail_full_url)

    res = requests.get(actor_detail_full_url)
    soup = BeautifulSoup(res.text, 'html.parser')

    for li_tag in soup.select('ul.default_info li'): # a 태그 안에 있는 li 태그들    
        #print(li_tag)
        #print(li_tag.text)

        #dict의 key에 해당하는 값을 추출한다. dict['직업']
        actor_item_field = li_tag.select_one('span.tit').text # 직업
        #print(actor_item_field) # 키 값 확인
        
        #dict의 value에 해당하는 값을 추출한다. dict['직업'] = '배우' 를 뽑아낸다.
        #<li><span class="tit">직업</span>배우</li>
        actor_item_value = re.sub('<span.*?>.*?</span>', '', str(li_tag))  # 값들 span 안에 있는 태그들 정규 표현식으로 파싱 / #<li>배우</li>
        actor_item_value = re.sub('<.*?>', '', actor_item_value) # 배우

        """ 적용 전
        \nhttps://www.facebook.com/cjesminjae
        """

        regex = re.compile(r'[\n\r\t]')
        actor_item_value = regex.sub('', actor_item_value)

        """ 적용 후
        https://www.facebook.com/cjesminjae
        """

        actor_item_dict[actor_item_field] = actor_item_value
        #print(actor_item_dict) # 누적되어 쌓임

    actor_item_list.append(actor_item_dict)

#actor_item_list

True


[{'배우이름': '강동원',
  '흥행지수': 159665,
  '출연영화': ['반도'],
  '랭킹': 1,
  '직업': '배우',
  '생년월일': '1981-01-18',
  '성별': '남',
  '홈페이지': 'http://www.gangdongwon.com',
  '신장/체중': '186cm, 68kg',
  '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)',
  '취미': '음악감상, 웹서핑, 운동, 신발 모으기',
  '특기': '축구'},
 {'배우이름': '이정현',
  '흥행지수': 139707,
  '출연영화': ['반도'],
  '랭킹': 2,
  '직업': '배우',
  '생년월일': '1980-02-07',
  '성별': '여',
  '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh',
  '신장/체중': '158cm, 38kg',
  '학교': '중앙대학교 연극영화',
  '취미': '바비 인형 수집, 피아노 연주',
  '특기': '별관찰, 시나리오 쓰기',
  '소속사': '에이바 앤 엔터테인먼트'},
 {'배우이름': '이레',
  '흥행지수': 119748,
  '출연영화': ['반도'],
  '랭킹': 3,
  '직업': '배우',
  '생년월일': '2006-03-12',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/leere.vv/'},
 {'배우이름': '권해효',
  '흥행지수': 99795,
  '출연영화': ['반도', '사이비', '한낮의 피크닉'],
  '랭킹': 4,
  '직업': '배우',
  '생년월일': '1965-11-06',
  '성별': '남',
  '신장/체중': '170cm, 62kg',
  '학교': '한양대학교 연극영화학 졸업'},
 {'배우이름': '구교환',
  '흥행지수': 79832,
  '출연영화': ['반도'],
  '랭킹':

#### 3단계
* 1, 2, 3 페이지의 21명 배우의 정보를 출력

In [105]:
actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict = dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2020-07'
formdata_dict['gender'] = 'all'

actor_item_list = list() # 21명 다 저장할 list

for page in range(1, 4): # 3페이지까지
    formdata_dict['page'] = page
    print('---> {} 페이지 출력됨'.format(page))

    res = requests.post(actor_url, data=formdata_dict) #post 방식에는 data속성에 넣어야함 / 통신을 위한 설정

    soup = BeautifulSoup(res.text, 'html.parser') # html 형식을 파싱할 준비 완료
    soup

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong') # '>' >> 바로 아래에 있다는 표시
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade') #print(len(rankings))

    #actor 변수가 <div class=name> 를 의미함 / 배우
    for index, actor in enumerate(actors):
        print('===> {} 번째 배우 출력됨'.format(index+1))
        actor_item_dict = dict()
        #print(actor)
        #유아인(2편) 에서 (2편)을 제거하고 저장한다.
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_item_dict['배우이름'] = actor_name

        #흥행지수 값에서 ,(콤마)를 제거하고 정수타입으로 변환해서 저장한다.
        actor_hit = int(hits[index].text.replace(',', '')) # replace() 함수 : ',' 를 빼주기 위함
        actor_item_dict['흥행지수'] = actor_hit
        #print(artor_name, actor_hit) 배우이름, 흥행지수

        #출연작
        movie_titles = movies[index].select('li a span')
        #출연작 여러개의 title을 저장하는 리스트
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_item_dict['출연영화'] = movie_title_list
        
        #순위
        actor_ranking = rankings[index].text
        actor_item_dict['랭킹'] = int(actor_ranking)
        #print(artor_name, actor_ranking)

    # ------------------------------------------------------------------------------------------
        #배우의 상세정보를 보기 위해 http get 요청 다시 요청함
        #('li.people_li div.name a')
        """
            <div class=name> = actors
                <a href="/db/person/info/?person_id=18040"> = actor_detail_url
        """
        actor_detail_url = actor.select_one('a').attrs['href'] #attrs[속성]
        actor_detail_full_url = urljoin(actor_url, actor_detail_url)
    #print(actor_detail_full_url)

        res = requests.get(actor_detail_full_url)
        soup = BeautifulSoup(res.text, 'html.parser')

        for li_tag in soup.select('ul.default_info li'): # a 태그 안에 있는 li 태그들    
            #print(li_tag)
            #print(li_tag.text)

            #dict의 key에 해당하는 값을 추출한다. dict['직업']
            actor_item_field = li_tag.select_one('span.tit').text # 직업
            #print(actor_item_field) # 키 값 확인
            
            #dict의 value에 해당하는 값을 추출한다. dict['직업'] = '배우' 를 뽑아낸다.
            #<li><span class="tit">직업</span>배우</li>
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(li_tag))  # 값들 span 안에 있는 태그들 정규 표현식으로 파싱 / #<li>배우</li>
            actor_item_value = re.sub('<.*?>', '', actor_item_value) # 배우

            """ 적용 전
            \nhttps://www.facebook.com/cjesminjae
            """

            regex = re.compile(r'[\n\r\t]')
            actor_item_value = regex.sub('', actor_item_value)

            """ 적용 후
            https://www.facebook.com/cjesminjae
            """

            actor_item_dict[actor_item_field] = actor_item_value
            #print(actor_item_dict) # 누적되어 쌓임

        actor_item_list.append(actor_item_dict)

print(len(actor_item_list))
actor_item_list

---> 1 페이지 출력됨
===> 1 번째 배우 출력됨
===> 2 번째 배우 출력됨
===> 3 번째 배우 출력됨
===> 4 번째 배우 출력됨
===> 5 번째 배우 출력됨
===> 6 번째 배우 출력됨
===> 7 번째 배우 출력됨
---> 2 페이지 출력됨
===> 1 번째 배우 출력됨
===> 2 번째 배우 출력됨
===> 3 번째 배우 출력됨
===> 4 번째 배우 출력됨
===> 5 번째 배우 출력됨
===> 6 번째 배우 출력됨
===> 7 번째 배우 출력됨
---> 3 페이지 출력됨
===> 1 번째 배우 출력됨
===> 2 번째 배우 출력됨
===> 3 번째 배우 출력됨
===> 4 번째 배우 출력됨
===> 5 번째 배우 출력됨
===> 6 번째 배우 출력됨
===> 7 번째 배우 출력됨
21


[{'배우이름': '강동원',
  '흥행지수': 159665,
  '출연영화': ['반도'],
  '랭킹': 1,
  '직업': '배우',
  '생년월일': '1981-01-18',
  '성별': '남',
  '홈페이지': 'http://www.gangdongwon.com',
  '신장/체중': '186cm, 68kg',
  '학교': '한양대 - 상명대학교대학원 연극영화 (석사과정 재학)',
  '취미': '음악감상, 웹서핑, 운동, 신발 모으기',
  '특기': '축구'},
 {'배우이름': '이정현',
  '흥행지수': 139707,
  '출연영화': ['반도'],
  '랭킹': 2,
  '직업': '배우',
  '생년월일': '1980-02-07',
  '성별': '여',
  '홈페이지': 'https://www.instagram.com/mermaidleejh/https://twitter.com/mermaidjh',
  '신장/체중': '158cm, 38kg',
  '학교': '중앙대학교 연극영화',
  '취미': '바비 인형 수집, 피아노 연주',
  '특기': '별관찰, 시나리오 쓰기',
  '소속사': '에이바 앤 엔터테인먼트'},
 {'배우이름': '이레',
  '흥행지수': 119748,
  '출연영화': ['반도'],
  '랭킹': 3,
  '직업': '배우',
  '생년월일': '2006-03-12',
  '성별': '여',
  '홈페이지': 'https://www.facebook.com/leere.vv/'},
 {'배우이름': '권해효',
  '흥행지수': 99795,
  '출연영화': ['반도', '사이비', '한낮의 피크닉'],
  '랭킹': 4,
  '직업': '배우',
  '생년월일': '1965-11-06',
  '성별': '남',
  '신장/체중': '170cm, 62kg',
  '학교': '한양대학교 연극영화학 졸업'},
 {'배우이름': '구교환',
  '흥행지수': 79832,
  '출연영화': ['반도'],
  '랭킹':

#### 4단계
* 1년치 데이터 1905명 배우의 정보를 출력

In [106]:
import requests
from bs4 import BeautifulSoup
import re
import pymongo
from urllib.parse import urljoin
from itertools import count # itertools 내부함수

actor_url = 'http://www.cine21.com/rank/person/content'

formdata_dict = dict()
formdata_dict['section'] = 'actor'
formdata_dict['period_start'] = '2019-07'
formdata_dict['gender'] = 'all'

actor_item_list = list() # 21명 다 저장할 list

#for page in range(1, 4): # 3페이지까지
for page in count(1): # 끝까지 돈다.
    formdata_dict['page'] = page
    print('---> {} 페이지 출력됨'.format(page))

    res = requests.post(actor_url, data=formdata_dict) #post 방식에는 data속성에 넣어야함 / 통신을 위한 설정

    soup = BeautifulSoup(res.text, 'html.parser') # html 형식을 파싱할 준비 완료
    soup

    actors = soup.select('li.people_li div.name')
    if len(actors) == 0:
        break
    hits = soup.select('ul.num_info > li > strong') # '>' >> 바로 아래에 있다는 표시
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade') #print(len(rankings))

    #actor 변수가 <div class=name> 를 의미함 / 배우
    for index, actor in enumerate(actors):
        actor_item_dict = dict()
        #print(actor)
        #유아인(2편) 에서 (2편)을 제거하고 저장한다.
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_item_dict['배우이름'] = actor_name
        print('===> {}({}) 배우 출력됨'.format(actor_name, len(actor_name)))

        #흥행지수 값에서 ,(콤마)를 제거하고 정수타입으로 변환해서 저장한다.
        actor_hit = int(hits[index].text.replace(',', '')) # replace() 함수 : ',' 를 빼주기 위함
        actor_item_dict['흥행지수'] = actor_hit
        #print(artor_name, actor_hit) 배우이름, 흥행지수

        #출연작
        movie_titles = movies[index].select('li a span')
        #출연작 여러개의 title을 저장하는 리스트
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_item_dict['출연영화'] = movie_title_list
        
        #순위
        actor_ranking = rankings[index].text
        actor_item_dict['랭킹'] = int(actor_ranking)
        #print(artor_name, actor_ranking)

    # ------------------------------------------------------------------------------------------
        #배우의 상세정보를 보기 위해 http get 요청 다시 요청함
        #('li.people_li div.name a')
        """
            <div class=name> = actors
                <a href="/db/person/info/?person_id=18040"> = actor_detail_url
        """
        actor_detail_url = actor.select_one('a').attrs['href'] #attrs[속성]
        actor_detail_full_url = urljoin(actor_url, actor_detail_url)
    #print(actor_detail_full_url)

        res = requests.get(actor_detail_full_url)
        soup = BeautifulSoup(res.text, 'html.parser')

        for li_tag in soup.select('ul.default_info li'): # a 태그 안에 있는 li 태그들    
            #print(li_tag)
            #print(li_tag.text)

            #dict의 key에 해당하는 값을 추출한다. dict['직업']
            actor_item_field = li_tag.select_one('span.tit').text # 직업
            #print(actor_item_field) # 키 값 확인
            
            #dict의 value에 해당하는 값을 추출한다. dict['직업'] = '배우' 를 뽑아낸다.
            #<li><span class="tit">직업</span>배우</li>
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(li_tag))  # 값들 span 안에 있는 태그들 정규 표현식으로 파싱 / #<li>배우</li>
            actor_item_value = re.sub('<.*?>', '', actor_item_value) # 배우

            """ 적용 전
            \nhttps://www.facebook.com/cjesminjae
            """

            regex = re.compile(r'[\n\r\t]')
            actor_item_value = regex.sub('', actor_item_value)

            """ 적용 후
            https://www.facebook.com/cjesminjae
            """

            actor_item_dict[actor_item_field] = actor_item_value
            #print(actor_item_dict) # 누적되어 쌓임

        actor_item_list.append(actor_item_dict)

print(len(actor_item_list))
actor_item_list

---> 1 페이지 출력됨
===> 이병헌(3) 배우 출력됨
===> 하정우(3) 배우 출력됨
===> 마동석(3) 배우 출력됨
===> 전혜진(3) 배우 출력됨
===> 이성민(3) 배우 출력됨
===> 김아중(3) 배우 출력됨
===> 박정민(3) 배우 출력됨
---> 2 페이지 출력됨
===> 강기영(3) 배우 출력됨
===> 조진웅(3) 배우 출력됨
===> 조정석(3) 배우 출력됨
===> 배수지(3) 배우 출력됨
===> 유아인(3) 배우 출력됨
===> 임윤아(3) 배우 출력됨
===> 고두심(3) 배우 출력됨
---> 3 페이지 출력됨
===> 장기용(3) 배우 출력됨
===> 권상우(3) 배우 출력됨
===> 곽도원(3) 배우 출력됨
===> 유해진(3) 배우 출력됨
===> 최유화(3) 배우 출력됨
===> 박인환(3) 배우 출력됨
===> 정해인(3) 배우 출력됨
---> 4 페이지 출력됨
===> 정유미(3) 배우 출력됨
===> 김지영(3) 배우 출력됨
===> 류준열(3) 배우 출력됨
===> 이봉련(3) 배우 출력됨
===> 조우진(3) 배우 출력됨
===> 공유(2) 배우 출력됨
===> 강동원(3) 배우 출력됨
---> 5 페이지 출력됨
===> 김종구(3) 배우 출력됨
===> 박지환(3) 배우 출력됨
===> 이정현(3) 배우 출력됨
===> 김상중(3) 배우 출력됨
===> 이희준(3) 배우 출력됨
===> 김미경(3) 배우 출력됨
===> 허성태(3) 배우 출력됨
---> 6 페이지 출력됨
===> 김병순(3) 배우 출력됨
===> 박신혜(3) 배우 출력됨
===> 염정아(3) 배우 출력됨
===> 배성우(3) 배우 출력됨
===> 이경영(3) 배우 출력됨
===> 허준호(3) 배우 출력됨
===> 권해효(3) 배우 출력됨
---> 7 페이지 출력됨
===> 황효은(3) 배우 출력됨
===> 이레(2) 배우 출력됨
===> 이하늬(3) 배우 출력됨
===> 김래원(3) 배우 출력됨
===> 공민정(3) 배우 출력됨
===>

ConnectionError: HTTPConnectionPool(host='www.cine21.com', port=80): Max retries exceeded with url: /db/person/info/?person_id=3747 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000011F3916AC18>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [107]:
print(len(actor_item_list))
# list comprehension >> 이상한 값들 거르기
actor_final_list = [actor for actor in actor_item_list if len(actor['배우이름']) < 10]
print(len(actor_final_list))

823
822


In [ ]:
import json

with open('data/')